In [ ]:
!pip install transformers

In [ ]:
!pip install pytorch-lightning

In [4]:
import numpy as np
import pandas as pd

import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

import re
import math
import random
from tqdm import tqdm

## (1) 데이터 전처리
문장부호 유지

In [5]:
df = pd.read_csv("/content/drive/MyDrive/데이터/인물 챗봇/선우_최종.csv")

In [6]:
df.shape

(24134, 4)

In [7]:
df.head(5)

,Unnamed: 0,Q,A,role
0,0,"야, 우유도!",먹어 니가 갖다 새끼야.,선우
1,1,"야, 우유도!",니가 갖다 새끼야. 먹어,선우
2,2,"야, 우유도!",니가 갖다 먹어 새끼야.,선우
3,3,전에처럼 우유 섞어서.,먹어 니가 갖다 새끼야.,선우
4,4,전에처럼 우유 섞어서.,니가 갖다 새끼야. 먹어,선우


In [8]:
df.drop(['Unnamed: 0', 'role'], inplace = True, axis = 1)

## (2) 모델링
GPU

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [10]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [11]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = "<usr>"
        self.a_token = "<sys>"
        self.bos = '</s>'
        self.eos = '</s>'
        self.pad = '<pad>'
        self.mask = '<mask>'
        self.tokenizer = tokenizer

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문 열을 가져온다.
        a = turn["A"]  # 답변 열을 가져온다.

        q_toked = self.tokenizer.tokenize(self.q_token + q  + self.bos)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        # 최대길이 - 질문길이
            if a_len <= 0:       # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   # 질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        # 질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [12]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data).to(device), torch.LongTensor(mask).to(device), torch.LongTensor(label).to(device)

In [13]:
train_set = ChatbotDataset(df, max_len=40) 
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [14]:
model  = model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [15]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18

In [16]:
for epoch in tqdm(range(epoch)):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-12-d9c5350f11d2>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.LongTensor(data).to(device), torch.LongTensor(mask).to(device), torch.LongTensor(label).to(device)
100%|██████████| 10/10 [45:21<00:00, 272.15s/it]

end


22분

In [17]:
torch.save(model, '/content/drive/MyDrive/데이터/인물 챗봇/선우kogpt2.pt')

##(3) 챗봇 결과
NONE

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 19.5 MB/s eta 0:00:00


In [2]:
import torch

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',pad_token='<pad>', mask_token='<mask>')
model = torch.load('/content/drive/MyDrive/데이터/인물 챗봇/선우kogpt2.pt', map_location=device)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [5]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode("<usr>" + q +"<sys>" + a)).unsqueeze(dim=0)
            input_ids.to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == '</s>':
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

user > 안녕
Chatbot > 안녕하세요. 예~
user > 뭐하고 있어
Chatbot > 난 허..그럼 뭘 한거야?
user > 학교가야지
Chatbot > 내일 잠깐 얼굴 볼까요? 저녁 시간에 학교 앞으로 갈게.
user > 왜?
Chatbot > 나 화이트 빌려줘. 좀 수정액.
user > 나도 없어
Chatbot > 학급비 너 오늘  얼마 받았다 그랬지?
user > 20만원 정도?
Chatbot > ᄒᄒ
user > 웃지마
Chatbot > 안돼, 너 그냥 자.
user > 그래 잘자
Chatbot > 아저씨한테 잘 니네 해. 아저씨 진짜예요,
user > 배고프다
Chatbot > 어. 엄마, 나 요 앞에 나갔다 올게. 금방 와요.
user > 재미없어
Chatbot > 어. 너 진짜 괜찮아.
user > 추워
Chatbot > 양말도 안 신고, 추워요?
user > 네 추워요
Chatbot > 양말도 안 신고, 추워요?
user > 양말이 없어
Chatbot > 양말도 안 신고, 안 해도 돼요!
user > 학주 만났어
Chatbot > 만났어. 저랑 6년
user > 이제 공부하자
Chatbot > 공부해 새끼야
user > 정환이 어때
Chatbot > 야. 나 화장실 좀. 내 방에 있어.
user > 진주는 잘 있어?
Chatbot > 진주야..!! 있었어? 잘
user > 떡볶이 먹고 싶어
Chatbot > 계세요!! 안녕히
user > 내일 뭐해
Chatbot > 내일은 세계사밖에 돌리고 저녁에 진주랑 놀기로 했어.
user > 좋아해
Chatbot > 장난 아닌 거.. 아시잖아요.
user > 사랑해
Chatbot > 갔다 엄마, 올게요..!!!
user > 학교가자
Chatbot > 내일 얼굴 볼까요? 저녁 시간에 학교 앞으로 갈게.
user > 뭐하려고
Chatbot > 우리 집 반상회 하네. 노래하는 거 보니까 거의 다 끝나간다.
user > 노래 잘 불러?
Chatbot > 노래는 듣냐?
user > 당연하지
Chatbot 